# NIA 과제 인도네시아어 말뭉치 LexisNexis Crawler

In [ ]:
#https://github.com/SergeyPirogov/webdriver_manager
%pip install selenium
%pip install webdriver_manager 

Note: you may need to restart the kernel to use updated packages.


In [7]:
#IMPORT LIBRARIES

from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common import by
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

import time
import re
import os
import shutil
import math
import sys

In [13]:
#DEFINE FUNCTIONS

def initiate_driver(browsertype, download_path):
    global driver
    global wait
    global actionChains

    service = Service()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    capabilities = {
    'browserName': 'chrome',
    'chromeOptions':  {
        'useAutomationExtension': False,
        'forceDevToolsScreenshot': True,
        'args': ['--start-maximized', '--disable-infobars']
    }
    } 
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_settings.popups" : 0,
        "profile.default_content_setting_values.automatic_downloads" : 1,
        "download.prompt_for_download": 'False',
        "download.default_directory" : download_path,
        "download.directory_upgrade": True,
        "safebrowsing.1enabled": True
        }
    chrome_options.add_experimental_option("prefs", prefs)

    if browsertype == 'chromium':
        driver = webdriver.Chrome(executable_path='/snap/bin/chromium.chromedriver', options=chrome_options)
    elif browsertype == 'chrome':
        driver = webdriver.Chrome(service=service, options=chrome_options)

    wait = WebDriverWait(driver, 300)
    actionChains = ActionChains(driver)
    driver.maximize_window()

#로그인 자동화
def login(id, pw) :
    driver.find_element(By.XPATH, '//*[@id="userid"]').send_keys(id)
    driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(pw)
    driver.implicitly_wait(1)
    driver.find_element(By.XPATH,'//input[@id="signInSbmtBtn"]').send_keys(Keys.ENTER)
    time.sleep(10)
    print('Finish login')

#다음 화면으로 넘어갈 때 까지 대기
def waitforspinner():
    wait.until(EC.invisibility_of_element_located((By.XPATH, '//div[@class="box spinner-interactive"]')))

#홈페이지 열기
def open_url(url):
    time.sleep(20)
    driver.get(url)

#캡션 선택
def click_link(navigate_links):
    for link_caption in navigate_links:
        print('Clicking ' + link_caption)
        linkje = wait.until(EC.element_to_be_clickable((By.XPATH, f'//a[@data-text="{link_caption}"]')))
        linkje.click()

def select_source(sources):
    if media == [] or  media[0] == 'all':
        pass
    else:
        print('Selecting source button')
        wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@name='allsources']")))
        driver.find_element_by_xpath("//input[@name='allsources'][@ng-reflect-value='false']").click()
        time.sleep(1)
        if driver.find_element(By.XPATH, "//input[@name='allsources'][@ng-reflect-value='false']").is_selected() == False:
            driver.find_element(By.XPATH, "//input[@name='allsources'][@ng-reflect-value='false']").click()
        for source in sources:
            print('Selecting ' + source)
            linkje = wait.until(EC.element_to_be_clickable((By.XPATH, f"//input[@aria-label='{source}']")))
            linkje.send_keys(Keys.SPACE)

def enter_query(keywords):
    print('Entering Query: ' + keywords)
    driver.find_element(By.XPATH,"//*[@id='searchTerms']").send_keys(keywords)
    driver.implicitly_wait(1)
    driver.find_element(By.XPATH,"//*[@id='searchTerms']").send_keys(Keys.ENTER)
    driver.implicitly_wait(2)
    print('Waiting for results to load')

#인도네시아어 선택
def filter_language(language):
    lan_btn = '//button[@aria-label="Expand Language"]'
    wait.until(EC.presence_of_element_located((By.XPATH, lan_btn)))
    time.sleep(5)
    driver.execute_script("arguments[0].click();",driver.find_element(By.XPATH, lan_btn))
    time.sleep(5)
    print('Indonesian selected')

    #more_btn
    try:
        driver.find_element(By.XPATH, '//div[@class="lessmore"]/button').click()
    except:
        pass

    wait.until(EC.element_to_be_clickable((By.XPATH, f"//label[@title='{language}']")))
    # 인도네시아 선택 버튼 클릭 5번 반복실행
    for _ in range(5):
        driver.find_element(By.XPATH, f"//label[@title='{language}']").click()
        time.sleep(5)  # 5번의 각 클릭 사이마다 5초 휴식

    close_btn = '//button[@aria-label="Collapse Language"]'
    wait.until(EC.presence_of_element_located((By.XPATH, close_btn)))
    driver.execute_script("arguments[0].click();",driver.find_element(By.XPATH, close_btn))
    driver.implicitly_wait(5)
    waitforspinner()
    
    #뉴스 카테고리 선택
def select_category():
    driver.find_element(By.XPATH, '//*[@id="startincontentbutton"]').click()
    driver.find_element(By.XPATH, '//*[@id="kwkmk"]/header/div/div/span/div/div[2]/ul/li[8]/button/span/span[1]').click()
    time.sleep(5)
    print("News category selected")

def filter_publication(type_list):
    print('publication type : ' , type_list)
    pub_btn = '//button[@aria-label="Expand Publication Type"]'
    driver.execute_script("arguments[0].click();",driver.find_element_by_xpath(pub_btn))
    time.sleep(1)

    for type in type_list:
        linkpub = wait.until(EC.element_to_be_clickable((By.XPATH, f"//label[@title='{type}']")))
        linkpub .send_keys(Keys.SPACE)
    
    close_btn = '//button[@aria-label="Collapse Publication Type"]'
    wait.until(EC.presence_of_element_located((By.XPATH, close_btn)))
    driver.execute_script("arguments[0].click();",driver.find_element_by_xpath(close_btn))
    driver.implicitly_wait(3)
    waitforspinner()

    #날짜 지정
def filter_date(start_date, end_date):
    tl_btn = '//button[@aria-label="Expand Timeline"]'
    wait.until(EC.presence_of_element_located((By.XPATH, tl_btn)))
    driver.execute_script("arguments[0].click();",driver.find_element(By.XPATH, tl_btn))
    time.sleep(10)

    #시작년도
    st_date = driver.find_element(By.XPATH, '//div[@class="date-form-group"]/div[1]/datepicker/div/input')
    for x in range(10): #clear()와Keys.CONTRAL + "a"작동이 안됨. for문을 통해 백스페이스로 지우기
        st_date.send_keys(Keys.BACKSPACE)

    st_date.send_keys(start_date)
    #마지막년도
    ed_date = driver.find_element(By.XPATH, '//div[@class="date-form-group"]/div[2]/datepicker/div/input')

    for x in range(10):
        ed_date.send_keys(Keys.BACKSPACE)
    ed_date.send_keys(end_date)
    ed_date.send_keys(Keys.TAB)

    #날짜설정 확인클릭
    dt_btn = driver.find_element(By.XPATH, '//div[@class="date-form-input"]/button')
    driver.execute_script("arguments[0].click();",dt_btn)
    driver.implicitly_wait(3)

    waitforspinner()
    print("Date filtered")

def sort_results(ranking): #Check amount of search results and sort from new to old
    waitforspinner()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="sortbymenulabel"]')))
    driver.find_element(By.XPATH, '//*[@id="sortbymenulabel"]').click()
    driver.implicitly_wait(1)

    editsort = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-action="editsort"]')))
    editsort.click()

    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]')))
    driver.find_element(By.XPATH, '//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]').click()
    driver.implicitly_wait(3)

    if ranking == 'newest':
        driver.find_element(By.XPATH, '//select[@id="sort-by-dropdown-2"]/option[text()="Date (newest-oldest)"]').click()
    elif ranking == 'oldest':
        driver.find_element(By.XPATH, '//select[@id="sort-by-dropdown-2"]/option[text()="Date (oldest-newest)"]').click()
    
    #Click save
    button_save = '//*[@id="saveChangesButton"]'
    wait.until(EC.element_to_be_clickable((By.XPATH, button_save)))
    driver.find_element(By.XPATH, button_save).click()
    
    waitforspinner()

def count_results():
    global last_page
    
    label_results = driver.find_element(By.XPATH, '//*[@id="content"]/header/h2/span').text
    print('Output ' + label_results)
    
    resultcount = int(re.sub('[^0-9]+', '', label_results))
    last_page = math.ceil(resultcount / 50)

    print(last_page)
    print('Found ' + str(resultcount) + ' articles on ' + str(last_page) + ' pages')

def filter_duplicates():
    print('Enabling duplicate filter')
    dropdown_duplicate = "//button[@class='current trigger collapsed icon la-TriangleDownAfter']"
    wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_duplicate)))
    driver.find_element(By.XPATH, dropdown_duplicate).click()
    driver.implicitly_wait(1)
    driver.find_element(By.XPATH, "//button[@data-value='moderate']").click()

    waitforspinner()

def selectallarticles():
    waitforspinner()
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@data-action="selectall"]')))
    driver.find_element(By.XPATH, '//*[@data-action="selectall"]').send_keys(Keys.SPACE)
    time.sleep(1)
    if driver.find_element(By.XPATH, '//*[@data-action="selectall"]').is_selected() == False:
        driver.find_element(By.XPATH, '//*[@data-action="selectall"]').send_keys(Keys.SPACE)
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@class="select more"]')))
    print("selected all")

def opendownloaddialog():
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@data-action="downloadopt"]')))
    driver.find_element(By.XPATH, '//*[@data-action="downloadopt"]').send_keys(Keys.SPACE)

def setdoctype(doctype):
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@data-action="download"]')))
    if doctype == 'docx':    
        driver.find_element(By.XPATH, '//*[@id="Docx"]').click() #select Docx format"
    elif doctype == 'rtf':
        driver.find_element(By.XPATH, '//*[@id="Rtf"]').click() #select Rtf format"
    elif doctype == 'pdf':
        driver.find_element(By.XPATH, '//*[@id="Pdf"]').click() #select Rtf format"

def clickdownload():
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@data-action="download"]')))
    driver.find_element(By.CSS_SELECTOR, 'body > aside > footer > div > button.button.primary').send_keys(Keys.SPACE)

def downloadf():
    selectallarticles()
    opendownloaddialog()
    
    global download_settings
    download_settings=True
    if download_settings is False:
    
        # Formatting options for LexisNexisTools
        driver.find_element(By.XPATH, '//*[@id="tab-FormattingOptions"]').click()
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="DisplayFirstLastNameEnabled"]').click()
        driver.find_element(By.XPATH, '//*[@id="EmbeddedReferences"]').click()

        download_settings = True
    clickdownload()

    try:
        #Wait for the job description box to appear
        wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="status-message success"]')))
        
        #Wait for job description dialog to disappear
        wait.until(EC.invisibility_of_element_located((By.XPATH, '//span[@class="status-message success"]')))
    except:
        driver.refresh()
        selectallarticles()
        opendownloaddialog()
        clickdownload()

    i = 1
    while not len([fname for fname in os.listdir() if re.match('Files\(\d+\).?[.DOCX]', fname)])!=0:
        i += 1
        time.sleep(0.5)
        if i > 50:
            print('keep going')
            break 

def start_downloading():
    global num
    for num in tqdm(range(0, (last_page))):
        time.sleep(1)
        downloadf()
        try:
            driver.find_element(By.XPATH, "//a[@data-action='nextpage']").click()
            waitforspinner()
        except:
            print('Seems like there are no more pages left')
            break
            
def select_50data():
    driver.find_element(By.XPATH, '//*[@id="sortbymenulabel"]').click()
    driver.implicitly_wait(3)
        
    driver.find_element(By.XPATH, '//*[@id="results-list-delivery-toolbar"]/div/ul[2]/li/div/div/button[6]/span').click()
    driver.implicitly_wait(3)
    
    driver.find_element(By.XPATH, '//*[@id="mainSettings"]/div[2]/div/div[2]/div[1]/settingsgeneral/div/section[2]/select/option[3]').click()
    driver.implicitly_wait(3)
    
    driver.find_element(By.XPATH, '//*[@id="saveChangesButton"]').click()
    driver.implicitly_wait(3)
    
    driver.find_element(By.XPATH, '//*[@id="sortbymenulabel"]').click()
    driver.find_element(By.XPATH, '//*[@id="results-list-delivery-toolbar"]/div/ul[2]/li/div/div/button[5]').click()

#ACTION PARAMETERS
start_page = "https://advance.lexis.com" #Open url
second_page = "https://advance.lexis.com/search/?pdmfid=1000516&crid=a41fdcd5-3aeb-4988-ad79-e6ae28a5c878&pdsearchterms=Lovelyz&pdtypeofsearch=searchboxclick&pdsearchtype=SearchBox&pdstartin=&pdpsf=&pdqttype=and&pdquerytemplateid=&ecomp=yd1vkkk&earg=pdsf&prid=38726445-36c3-4e06-8848-988ffadd1419" 
ID = 'minwoo@arspraxia.com' #Enter ID & Password
Password='treum8864!'
start_date = "01/01/2013" #Set up date for filtering 
end_date = "05/01/2023"
language='Indonesian' #Set up language to Indonesian
download_path= 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\20230927_data\\data_Lovelyz' #Set up download path

def automation():
    initiate_driver('chrome', download_path) #Type either chrome or chromium depending on webdriver
    open_url(start_page) #홈페이지 열기
    login(ID, Password) #자동 로그인
    open_url(second_page) #두번째 페이지 창 열기
    time.sleep(3)
    select_category() #카테고리 선택
    filter_date(start_date, end_date) #날짜 지정
    filter_language(language) #언어 선택
    time.sleep(10)
    count_results() #페이지 문서 전체 선택
    start_downloading() #선택한 문서 다운로드

### 20230802 최신화 => 정상 작동중
### 20230927 최신화

In [12]:
#START AUTOMATION
automation()

Finish login
News category selected
Date filtered
Indonesian selected
Output News (400)
8
Found 400 articles on 8 pages


  0%|          | 0/8 [00:00<?, ?it/s]

selected all
oh oh not good


 12%|█▎        | 1/8 [00:54<06:20, 54.37s/it]

selected all
oh oh not good


...

 75%|███████▌  | 6/8 [05:49<01:56, 58.36s/it]

selected all
oh oh not good


 88%|████████▊ | 7/8 [06:48<00:58, 58.70s/it]

selected all
oh oh not good


100%|██████████| 8/8 [07:48<00:00, 58.51s/it]


In [16]:
#tqdm이 먼저 끝날 시 사용
count_results()
start_downloading()

Output News (758)
16
Found 758 articles on 16 pages


  0%|          | 0/16 [00:00<?, ?it/s]

selected all
oh oh not good


  6%|▋         | 1/16 [00:58<14:37, 58.47s/it]

selected all
oh oh not good


...

selected all
oh oh not good


 38%|███▊      | 6/16 [06:46<10:55, 65.50s/it]

selected all
oh oh not good


 44%|████▍     | 7/16 [07:41<09:17, 61.97s/it]

selected all
